<center><h1>Paper results - (3º Notebook)</h1></center>

##### Experiments:

1. Comparing learning approaches: Global x Local x Regional
 1.1 Comparing boxplots of evaluation metrics
 1.2 Eigenvalues analysis
 
##### Output:
 - **Paper (Table 5):** Performance in pk of G/L/R-LSSVM experiment.
 - **Paper (Table 6):** Performance in Vertebral Column of G/L/R-LSSVM experiment.
 - **Paper (Table 7):** Performance in Wall-following of G/L/R-LSSVM experiment.
 
 - **Paper (Figure 8):** Train and test accuracy distributions in G/L/R-LSSVM experiment.

### 1. Comparing learning approaches: Global x Local x Regional  <a class="anchor" id="r-lssvm"></a>

Loading results:

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from devcode.utils import load_csv_as_pandas
from devcode.utils.evaluation import cm2f1, cm2acc, cm2sen, cm2esp

df = {
    'global'    : load_csv_as_pandas(path="results/local-results/cbic/temp_glssvm_cbic"),
    'local'     : load_csv_as_pandas(path="results/local-results/cbic/temp_llssvm_cbic/results"),
    'regional'  : load_csv_as_pandas(path="results/regional-results/temp_rlssvm_somfix/results")
}

datasets = np.unique(df['global']['dataset_name'].values).tolist()

Dataset:  Features.shape:   # of classes:
vc2c      (310, 6)          2
vc3c      (310, 6)          3
wf24f     (5456, 24)        4
wf4f      (5456, 4)         4
wf2f      (5456, 2)         4
pk        (195, 22)         2


#### 1.1 Comparing boxplots of evaluation metrics <a class="anchor" id="boxplot_tr_ts"></a>

In [3]:
import plotly.offline as py
import plotly.graph_objs as go

from IPython.core.display import display, HTML

py.init_notebook_mode(connected=True) # enabling plot within jupyter notebook

set_dict = {'treino': 'cm_tr', 'teste': 'cm_ts'}

for dataset_name in datasets:
    display(HTML('<center><h1>'+dataset_name+'</h1></center>'))
    
    data = {}
    for classifier in df.keys():
        df_dataset = df[classifier].loc[df[classifier]['dataset_name'] == dataset_name]
        
        n_exp = len(df_dataset)
        data[classifier] = {
            'treino': [None]*n_exp, 
            'teste' : [None]*n_exp
        }

        for set_ in set_dict:
            cm_series = df_dataset[set_dict[set_]].values
            
            for i in range(n_exp):
                temp_cm = np.frombuffer(eval(cm_series[i]), dtype='int64')
                cm = temp_cm.reshape( int( len(temp_cm)**(1/2) ) ,-1)
                
                # acuracia:
                data[classifier][set_][i] = cm2acc(cm)*100
                
    boxs = []
    for set_ in set_dict:
        for classifier in df.keys():
            cor = {
                'global':   "rgba(44, 160, 101, 0.5)",
                'local':    "rgba(93, 164, 214, 0.5)",
                'regional': "rgba(155, 89, 182,1.0)"
            }
            boxs.append(
                go.Box(
                    y = data[classifier][set_],
                    x = ["{}{} {}-LSSVM".format(set_[0].upper(), set_[1:],
                                                  classifier[0].upper())
                        ]*len(data[classifier][set_]),
                    name = "{}-LSSVM".format(classifier[0].upper()),
                    boxmean='sd',
                    marker_color=cor[classifier],
#                     showlegend = False if set_=='treino' else True
                    )
            )
    


    layout = go.Layout(
#         title = "Acurácia nos conjuntos de treino e teste [<b>{}</b>]".format(dataset_name),
        yaxis=dict(title="Acurácia (%)"),
        showlegend=False,
        legend=dict(x=.875, y=1)
    )

    fig = go.Figure(data=boxs,layout=layout)
    width = 2
    fig.add_trace(go.Scatter(
        x=['Treino G-LSSVM', 'Treino L-LSSVM', 'Treino R-LSSVM', 'Treino G-LSSVM'],
        y=[np.mean(data['global']['treino']), np.mean(data['local']['treino']),
           np.mean(data['regional']['treino']), np.mean(data['global']['treino'])],
        mode='lines+markers',
        line=dict(
            color="RoyalBlue",
            dash="dashdot",
#             dash="dot",
            width=width
        ),
        showlegend=False
    ))
    
    fig.add_trace(go.Scatter(
        x=['Teste G-LSSVM', 'Teste L-LSSVM', 'Teste R-LSSVM', 'Teste G-LSSVM'],
        y=[np.mean(data['global']['teste']), np.mean(data['local']['teste']),
           np.mean(data['regional']['teste']), np.mean(data['global']['teste'])],
        mode='lines+markers',
        line=dict(
            color="RoyalBlue",
            dash="dashdot",
#             dash="dot",
            width=width
        ),
        showlegend=False
    ))
    
    fig.update_layout(
        margin=dict(l=20, r=5, t=5, b=20),
#         paper_bgcolor="LightSteelBlue",
    )
    
    fig.show()
    
    fig.write_image("images/r-lssvm_{}.pdf".format(dataset_name))
    
    display(HTML('<hr>'))

#### 1.2 Eigenvalues analysis <a class="anchor" id="r-lssvm_eigenvalues"></a>

In [4]:
from devcode.analysis.eigenvalues import eigenvalues_analysis_regional

df_results = pd.read_csv("results/regional-results/ROLS - all - n_res=100 - 2019-07-10.csv")
eigenvalues_analysis_regional(datasets, df["regional"])

 
pk: 50 runs. 
case: k_opt=2; gamma=1.00E+00; sigma=3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 8.04E+01
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=3.16E-01 [9 instances]
dtypes = ['float64']
Worst conditioning: 1.10E+02
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [5 instances]
dtypes = ['float64']
Worst conditioning: 2.18E+06
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 2.81E+06
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [2 instances]
dtypes = ['float64']
Worst conditioning: 1.71E+10
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.28E+06
----------------------------------

 
vc2c: 50 runs. 
case: k_opt=2; gamma=1.00E+00; sigma=3.16E-01 [4 instances]
dtypes = ['float64']
Worst conditioning: 7.02E+05
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+00; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 2.17E+12
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.34E+05
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [4 instances]
dtypes = ['float64']
Worst conditioning: 1.10E+14
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=2.37E+00 [3 instances]
dtypes = ['float64']
Worst conditioning: 1.01E+14
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [6 instances]
dtypes = ['complex128']
Worst conditioning: 2.57E+19
-----------------------------

 
vc3c: 50 runs. 
case: k_opt=2; gamma=1.00E+02; sigma=3.16E-01 [1 instances]
dtypes = ['float64']
Worst conditioning: 6.49E+04
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [7 instances]
dtypes = ['float64']
Worst conditioning: 1.52E+14
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=2.37E+00 [3 instances]
dtypes = ['float64']
Worst conditioning: 4.33E+13
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.78E+01 [6 instances]
dtypes = ['complex128']
Worst conditioning: 6.07E+19
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+04; sigma=1.33E+02 [2 instances]
dtypes = ['complex128']
Worst conditioning: 6.13E+19
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 1.50E+12
--------------------------

 
wf24f: 50 runs. 
case: k_opt=2; gamma=1.00E+02; sigma=2.37E+00 [23 instances]
dtypes = ['float64']
Worst conditioning: 1.87E+12
-------------------------------------------------------


case: k_opt=2; gamma=1.00E+06; sigma=1.78E+01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.90E+17
-------------------------------------------------------


case: k_opt=3; gamma=1.00E+02; sigma=2.37E+00 [17 instances]
dtypes = ['float64']
Worst conditioning: 2.18E+12
-------------------------------------------------------


case: k_opt=4; gamma=1.00E+02; sigma=2.37E+00 [4 instances]
dtypes = ['float64']
Worst conditioning: 2.96E+11
-------------------------------------------------------


case: k_opt=5; gamma=1.00E+02; sigma=2.37E+00 [2 instances]
dtypes = ['float64']
Worst conditioning: 1.99E+11
-------------------------------------------------------


case: k_opt=7; gamma=1.00E+02; sigma=2.37E+00 [1 instances]
dtypes = ['float64']
Worst conditioning: 5.25E+11
--------------------------

 
wf2f: 50 runs. 
case: k_opt=5; gamma=1.00E+06; sigma=3.16E-01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 1.28E+36
-------------------------------------------------------


case: k_opt=6; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.02E+34
-------------------------------------------------------


case: k_opt=9; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 2.71E+34
-------------------------------------------------------


case: k_opt=12; gamma=1.00E+04; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 9.85E+33
-------------------------------------------------------


case: k_opt=12; gamma=1.00E+06; sigma=3.16E-01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 3.59E+34
-------------------------------------------------------


case: k_opt=13; gamma=1.00E+04; sigma=3.16E-01 [2 instances]
dtypes = ['complex128']
Worst conditioning: 2.23E+36
-----------

 
wf4f: 50 runs. 
case: k_opt=2; gamma=1.00E+04; sigma=3.16E-01 [7 instances]
dtypes = ['complex128']
Worst conditioning: 8.46E+33
-------------------------------------------------------


case: k_opt=4; gamma=1.00E+04; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 2.44E+31
-------------------------------------------------------


case: k_opt=5; gamma=1.00E+02; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.23E+33
-------------------------------------------------------


case: k_opt=5; gamma=1.00E+04; sigma=3.16E-01 [6 instances]
dtypes = ['complex128']
Worst conditioning: 8.46E+33
-------------------------------------------------------


case: k_opt=7; gamma=1.00E+04; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 4.82E+32
-------------------------------------------------------


case: k_opt=7; gamma=1.00E+06; sigma=3.16E-01 [1 instances]
dtypes = ['complex128']
Worst conditioning: 1.41E+34
--------------